In [7]:
from praatio import textgrid

In [2]:
input = "/Users/joregan/Desktop/"

In [8]:
from pathlib import Path

input_path = Path(input)
IS_DIR = False
if input_path.is_dir():
    IS_DIR = True
assert input_path.exists()

In [29]:
import re

def get_durations(filename):
    tg = textgrid.openTextgrid(filename, includeEmptyIntervals=False)

    if len(tg.tierNames) == 1:
        tier = tg.getTier(tg.tierNames[0])
    elif "whisperx" in tg.tierNames:
        tier = tg.getTier("whisperx")
    elif "utterances" in tg.tierNames:
        tier = tg.getTier("utterances")
    elif "words" in tg.tierNames:
        tier = tg.getTier("words")
    else:
        print("Be careful: file", filename, "has none of the expected tier names")

    longest_utt = 0.0
    longest_sil = 0.0
    shortest_utt = 100.0
    shortest_sil = 100.0

    def silence_like(text):
        text = text.strip()

        if text == "":
            return True
        if text in ["sp", "sil", "<eps>"]:
            return True
        m = re.match("^\[[^]]+\]$", text)
        if m:
            return True
        return False        

    silences = []
    utterances = []

    for interval in tier.entries:
        start = interval[0]
        end = interval[1]
        text = interval[2]
        duration = end - start

        text = text.strip()
        if silence_like(text):
            silences.append(duration)
            if duration > longest_sil:
                longest_sil = duration
            if duration < shortest_sil:
                shortest_sil = duration
        else:
            utterances.append(duration)
            if duration > longest_utt:
                longest_utt = duration
            if duration < shortest_utt:
                shortest_utt = duration
    
    return {
        "longest_utt": longest_utt,
        "longest_sil": longest_sil,
        "shortest_utt": shortest_utt,
        "shortest_sil": shortest_sil,
        "utterances": utterances,
        "silences": silences
    }
            

In [31]:
get_durations("/Users/joregan/Desktop/hsi_5_0718_227_001_main.TextGrid")

{'longest_utt': 26.394918272118844,
 'longest_sil': 1.3282541087066875,
 'shortest_utt': 0.38482016812781694,
 'shortest_sil': 0.12899471012678987,
 'utterances': [0.6774095954404107,
  0.803593735767544,
  3.699003293778354,
  1.0360382047912111,
  4.205563005973971,
  0.4449651264167471,
  2.9529563320441525,
  2.9952130151335865,
  0.3984762326119977,
  16.7262792675169,
  23.191480298177567,
  13.71676358035188,
  0.624279431092134,
  8.220721115012026,
  0.5379429140262175,
  3.2608638368749325,
  4.316825853296734,
  7.636137620243204,
  25.035160050482403,
  11.622581659972184,
  23.111419406973027,
  8.556231510282174,
  4.1924508971039245,
  3.566351070347423,
  20.879634105461918,
  0.7551734126418523,
  1.4206518988779635,
  0.8287480655735067,
  6.621972040021376,
  1.4831776361396578,
  8.11214465244285,
  6.2308245652138226,
  9.577608920103842,
  11.941110148946905,
  2.2076525434702603,
  17.931354553161555,
  11.423448638176296,
  8.493467332429987,
  10.69264642416595

In [24]:
import re

text1 = "[yes]"
text2 = "no"
text3 = "[like]yes,but[no]"

m = re.match("^\[[^]]+\]$", text3)
m
